<a href="https://colab.research.google.com/github/kanish-dev/All-AI-Agents/blob/main/Multi_Agent_System_using_Gemini_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U google-generativeai

In [29]:
import os
from google.colab import userdata
import google.generativeai as genai


def configure():
  secret_name = "GOOGLE_API_KEY"

  api_key = userdata.get(secret_name)
  if not api_key:
    raise ValueError(f"Secret '{secret_name}' not found. please check")

  genai.configure(api_key=api_key)
  return genai.GenerativeModel('models/gemini-2.5-flash')

In [ ]:
for m in genai.list_models():
    print(f"Model Name: {m.name}")
    print(f"Description: {m.description}")
    print(f"Input Modalities: {m.supported_generation_methods}")
    print("-" * 20)

Model Name: models/embedding-gecko-001
Description: Obtain a distributed representation of a text.
Input Modalities: ['embedText', 'countTextTokens']
--------------------
Model Name: models/gemini-2.5-pro-preview-03-25
Description: Gemini 2.5 Pro Preview 03-25
Input Modalities: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
--------------------
Model Name: models/gemini-2.5-flash-preview-05-20
Description: Preview release (April 17th, 2025) of Gemini 2.5 Flash
Input Modalities: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
--------------------
Model Name: models/gemini-2.5-flash
Description: Stable version of Gemini 2.5 Flash, our mid-size multimodal model that supports up to 1 million tokens, released in June of 2025.
Input Modalities: ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
--------------------
Model Name: models/gemini-2.5-flash-lite-preview-06-17
Description: Preview rel

### **AGENT 1 : THE PLANNER**

In [ ]:
def planner_agent(model, topic: str) ->list[str]:
  print('Planner Agent: Creating a plan...')
  prompt = f"""
  You are an expert research planner. Yout task is to break down the following topic
  into 3-5 specific, answerable questions. Return these questions as Python list of strings.

  TOPIC : "{topic}"

  Example output : ["question 1","question 2","question 3"]
  """
  try :
    response = model.generate_content(prompt)
    plan_str = response.text.strip().replace('[','').replace(']','').replace('"','')
    plan = [q.strip() for q in plan_str.split(',') if q.strip()]

    print('Plan created:')
    for i,q in enumerate(plan,1):
      print(f'{i}.{q}')
    return plan
  except Exception as e:
    print(f'Error in Planner Agent: {e} ')
    return []


## **AGENT 2: THE SEARCH AGENT**

In [ ]:
def search_agent(model, question: str) -> str:
    print(f"Search Agent: Researching question: '{question}'...")
    try:
        search_tool = genai.protos.Tool(
            google_search_retrieval=genai.protos.GoogleSearchRetrieval()
        )
        prompt = f"Provide a detailed answer to the following question: {question}"
        response = model.generate_content(prompt, tools=[search_tool])

        print("   - Information found.")
        return response.text
    except Exception as e:
        print(f"Error in Search Agent: {e}")
        return ""

### **AGENT 3 : THE SYNTHESISZER**

In [ ]:
def synthesizer_agent(model, topic: str, research_results: list) ->str:
  print("Synthesizer Agent: Writing the final report...")

  reserach_notes = ""
  for question, data in research_results:
    reserach_notes += f"### Question: {question}\n### Research Data:\n{data}\n\n===\n\n"

  prompt = f"""
  You are an expert research analyst. Your task is to synthesize the provided research notes
  into a comprehensive, well-structured reporton the topic: "{topic}".

  The report should contain an introduction, a Body that covers the key findings from the notes,
  and a conclusion. Use the information from the research notes ONLY.

  ## Research Notes ##
  {reserach_notes}
  """
  try:
    response = model.generate_content(prompt)
    return response.text
  except Exception as e:
    print(f"Error in Synthesizer Agent: {e}")
    return "Error: Could not genrate the final report."

### **The Conductor: The main() Orchestrator**

In [ ]:
def main():
  try:
    model = configure()
  except ValueError as e:
    print(e)
    return
  print("\nHello! I am your AI Research Assistant.")
  topic = input(" What topic would you like me to research today? ")

  if not topic.strip():
    print("A topic is requried to begin the research. Exiting.")
    return

  print(f"\nStrarting research process for: '{topic}'")

  research_plan = planner_agent(model,topic)
  if not research_plan:
    print("Could not create a research plan. Exiting")
    return

  research_results = []
  for question in research_plan:
    research_data = search_agent(model,question)
    if research_data:
      research_results.append((question, research_data))

  if not research_results:
    print("Could not find any information during the research. Exiting")
    return

  final_report = synthesizer_agent(model,topic, research_results)

  print("\n\n--- FINAL RESEARCH REPORT ---")
  print(f"## TOPIC: {topic}\n")
  print(final_report)
  print("\n\n--- END OF REPORT ---")

main()



Hello! I am your AI Research Assistant.
 What topic would you like me to research today? Explain quantum computing in simple termsExplain quantum computing in simple terms

Strarting research process for: 'Explain quantum computing in simple termsExplain quantum computing in simple terms'
Planner Agent: Creating a plan...
Plan created:
1.```python

  What are the core differences between a quantum computer and a regular computer
2.in simple terms?
3.What are the key quantum principles
4.like superposition and entanglement
5.that allow quantum computers to work?
6.What kinds of problems are quantum computers expected to solve better than traditional computers?
7.Is quantum computing a reality today
8.and what are its current limitations?

```
Search Agent: Researching question: '```python

  What are the core differences between a quantum computer and a regular computer'...


Error in Search Agent: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Search Grounding is not supported.
Search Agent: Researching question: 'in simple terms?'...


Error in Search Agent: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Search Grounding is not supported.
Search Agent: Researching question: 'What are the key quantum principles'...


Error in Search Agent: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Search Grounding is not supported.
Search Agent: Researching question: 'like superposition and entanglement'...


Error in Search Agent: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Search Grounding is not supported.
Search Agent: Researching question: 'that allow quantum computers to work?'...


Error in Search Agent: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Search Grounding is not supported.
Search Agent: Researching question: 'What kinds of problems are quantum computers expected to solve better than traditional computers?'...


Error in Search Agent: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Search Grounding is not supported.
Search Agent: Researching question: 'Is quantum computing a reality today'...


Error in Search Agent: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Search Grounding is not supported.
Search Agent: Researching question: 'and what are its current limitations?

```'...


Error in Search Agent: 400 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Search Grounding is not supported.
Could not find any information during the research. Exiting
